# 2 MA Crypto Futures Strategy

This is a simple strategy based on 2 MAs for crypto futures.

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) { return false; }
// disable widget scrolling

<IPython.core.display.Javascript object>

In [2]:
import xarray as xr

import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.data as qndata


def load_data(period):
    return qndata.cryptofutures.load_data(tail=period)


def strategy(data):
    close = data.sel(field='close')
    ma_slow = qnta.lwma(close, 200).isel(time=-1)
    ma_fast = qnta.lwma(close, 50).isel(time=-1)
    return xr.where(ma_fast > ma_slow, 1, 0)


weights = qnbt.backtest(
    competition_type="cryptofutures",
    load_data=load_data,
    lookback_period=365,
    start_date='2014-01-01',
    strategy=strategy
)


Run last pass...
Load data...


100% (15272 of 15272) |##################| Elapsed Time: 0:00:00 Time:  0:00:00


Run pass...
Ok.
---
Run first pass...
Load data...


100% (12032 of 12032) |##################| Elapsed Time: 0:00:00 Time:  0:00:00


Run pass...
Ok.
---
Load full data...


...

100% (118112 of 118112) |################| Elapsed Time: 0:00:00 Time:  0:00:00


Check missed dates...
Ok.
Normalization...
Done.
Write output: /root/fractions.nc.gz
---
Analyze results...
Check...
Check missed dates...
Ok.
Check sharpe ratio.
Ok.
Check correlation.



WARNING! This strategy correlates with other strategies.


The number of systems with a larger Sharpe ratio and correlation larger than 0.8: 1
The max correlation value (with systems with a larger Sharpe ratio): 1.0
Current sharpe ratio(5y): 1.8684642979815371

---
Calc global stats...
---
Calc stats per asset...
Build plots...
---
Select the asset (or leave blank to display the overall stats):


interactive(children=(Combobox(value='', description='asset', options=('', 'BTC')), Output()), _dom_classes=('…

At this stage code is ready for submission. Just click on the submission button in your account page and we will evaluate your strategy live on our servers!

---
There is a fast(one-pass) implementation. Use can use this approach to prototype your strategy. Beware, it is easy to create a looking forward strategy which will produce irrelevant output. So, verify your strategy with `qnt.backtester` when you finish your prototype.

```python
import xarray as xr

import qnt.ta as qnta
import qnt.stats as qns
import qnt.data as qndata
import qnt.output as qnout

# load data
data = qndata.cryptofutures.load_data(min_date='2013-01-01')

#calc weights
close = data.sel(field='close')
ma_slow = qnta.lwma(close, 200)
ma_fast = qnta.lwma(close, 50)
weights = xr.where(ma_fast > ma_slow, 1, 0)
weights = qnout.clean(weights, data)

#check
qnout.check(weights, data)

# write results
qnout.write(weights)

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice('2014-01-01',None)))
stats.to_pandas().tail()
```